<a href="https://colab.research.google.com/github/ansonmiu0214/C490CW/blob/master/Sentence_Level_QE_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CO490 Coursework: Quality Estimation

__Team__
* Anson Miu (kcm116)
* Cheryl Chen (czc16)
* Clara Gila (acg116)

## Setup

In [0]:
# Imports
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sklearn
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Setup CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch_device = getattr(torch, str(device))
torch_device.empty_cache()
print(f'DEVICE={torch_device.get_device_name()}')
print(torch.cuda.memory_summary(device=device))
print(torch.cuda.get_device_properties(device))

## Utilities

### Loading Data

In [0]:
if not os.path.exists('enzh_data.zip'):
    !wget -O enzh_data.zip https://competitions.codalab.org/my/datasets/download/03e23bd7-8084-4542-997b-6a1ca6dd8a5f
    !unzip enzh_data.zip

TRAIN_EN = 'train.enzh.src'
TRAIN_ZH = 'train.enzh.mt'
TRAIN_SCORES = 'train.enzh.scores'
VAL_EN = 'dev.enzh.src'
VAL_ZH = 'dev.enzh.mt'
VAL_SCORES = 'dev.enzh.scores'
TEST_EN = 'test.enzh.src'
TEST_ZH = 'test.enzh.mt'

In [0]:
# Read from file

with open(TRAIN_EN) as f:
    train_en = f.readlines()
with open(TRAIN_ZH) as f:
    train_zh = f.readlines()
with open(TRAIN_SCORES) as f:
    train_scores = [float(score.strip()) for score in f]
with open(VAL_EN) as f:
    val_en = f.readlines()
with open(VAL_ZH) as f:
    val_zh = f.readlines()
with open(VAL_SCORES) as f:
    val_scores = [float(score.strip()) for score in f]
with open(TEST_EN) as f:
    test_en = f.readlines()
with open(TEST_ZH) as f:
    test_zh = f.readlines()

### Metrics

In [0]:
from scipy.stats import pearsonr

def RMSELoss(pred, target):
    is_numpy = isinstance(pred, np.ndarray) or isinstance(target, np.ndarray)

    mean = np.mean if is_numpy else torch.mean
    sqrt = np.sqrt if is_numpy else torch.sqrt
    return sqrt(mean((pred - target) ** 2))

def pearson_score(pred, target):
    if isinstance(pred, torch.Tensor):
        pred = pred.numpy()

    if isinstance(target, torch.Tensor):
        target = target.numpy()

    score, _ = pearsonr(pred, target)
    return score

from sklearn.metrics import make_scorer
pearson_sklearn = make_scorer(pearson_score, greater_is_better=True)

### Data Processing

In [0]:
def concat_zip(en_embs, zh_embs):
    """Zip up the sentence embeddings as pairs."""

    if isinstance(en_embs, list):
        en_embs = torch.cat(en_embs)
    if isinstance(zh_embs, list):
        zh_embs = torch.cat(zh_embs)
    return torch.cat((en_embs, zh_embs), dim=1)


class SourceMTDataset(Dataset):
    """PyTorch dataset for **labelled** data."""

    def __init__(self, inputs, output):
        assert isinstance(inputs, list), f'Expected type of inputs to be list, got {type(inputs)}'

        self.inputs = inputs
        self.output = output

    def __getitem__(self, index):
        return [input_[index] for input_ in self.inputs], self.output[index]

    def __len__(self):
        return len(self.output)

def build_dataset(*, X, Y, idxs=None):
    """Return labelled Dataset object compatible with DataLoader.
    
    Optionally pass in `idxs` to filter which data points to use."""

    if idxs is not None:
        X = [input_[idxs] for input_ in X]
        Y = Y[idxs]

    Cnstr = torch.LongTensor if type(X[0]) == np.ndarray else torch.FloatTensor

    inputs_tensors = [Cnstr(input_) for input_ in X]
    return SourceMTDataset(inputs_tensors, Y)


class SourceMTTestset(Dataset):
    """PyTorch dataset for **unlabelled** data."""

    def __init__(self, inputs):
        assert isinstance(inputs, list), f'Expected type of inputs to be list, got {type(inputs)}'

        self.inputs = inputs

    def __getitem__(self, index):
        return [input_[index] for input_ in self.inputs]

    def __len__(self):
        return len(self.inputs[0])

def build_test_dataset(X):
    """Return unlabelled Dataset object compatible with DataLoader."""

    Cnstr = torch.LongTensor if type(X[0]) == np.ndarray else torch.FloatTensor

    inputs_tensors = [Cnstr(input_) for input_ in X]
    return SourceMTTestset(inputs_tensors)

def save_predictions(results, *, zip_name):
    """Save predictions into compressed text file for CodaLab submission."""

    with open('predictions.txt', 'w') as f:
        for line in results:
            f.write(f'{line}\n')

    zip_name = f'{zip_name}.zip'
    !zip $zip_name predictions.txt 

### Model Training

In [0]:
from pprint import PrettyPrinter

pp = PrettyPrinter()

def suppress_log(*args, **kwargs):
    pass

def debug_log(*args, **kwargs):
    print('[Debug]:', end='')
    print(*args, **kwargs)

In [0]:
def torch_model_factory(model_cls, model_params, *,
                        opt, lr, num_epochs, batch_size, loss_fn,
                        detailed_log=True):
    """A pseudo-decorator for a PyTorch model to support k-fold cross validation.

    Params:
        model_cls: Model class
        model_params: Parameters to construct `model_cls`

        others: for model training
    Returns:
        A factory function that builds a new model with additional attributes
    """

    def factory():
        model = model_cls(**model_params)
    
        def fit(dataset):
            """Performs mini-batch SGD and returns the loss per batch."""

            # Enter train mode
            model.train()
            model.to(device)

            # Construct data loader
            loader = DataLoader(dataset=dataset, batch_size=batch_size)

            optimiser = opt(model.parameters(), lr=lr)
            
            losses = []
            epoch_progress = range(1, num_epochs + 1)
            if not detailed_log:
                epoch_progress = tqdm(epoch_progress, desc='Epoch')

            for epoch in epoch_progress:
                if detailed_log:
                    header = f'Epoch {epoch}'
                    print(header)
                    print('=' * len(header))

                progress = loader
                if detailed_log:
                    progress = tqdm(progress,
                                    desc=f'Training: Mini-Batch SGD (size={batch_size})')
                for X, scores in progress:
                    optimiser.zero_grad()

                    pred = model(*(x.to(device) for x in X)).squeeze()
                
                    loss = loss_fn(pred, scores.to(device))

                    loss.backward()
                    optimiser.step()
                
                    losses.append(loss.cpu().detach().numpy())
                
                if detailed_log:
                    print(f'Most recent loss:', loss.cpu().detach().numpy())
                    print()
            return losses

        def predict(dataset, **metrics):
            """Returns the loss and metrics from the predictions."""

            # Enter evaluation mode
            model.eval()
            model.to(device)

            loader = DataLoader(dataset=dataset, batch_size=1)

            preds = []
            scores = []
            with torch.no_grad():
                for X, score in tqdm(loader, desc='Validating'):
                    pred = model(*(x.to(device) for x in X)).squeeze().cpu()
                    preds.append(pred)
                    scores.append(score)
            
                preds = torch.stack(preds)
                scores = torch.cat(scores)

                loss = loss_fn(preds, scores)
                metric_scores = {name: metric_fn(preds, scores)
                            for name, metric_fn in metrics.items()}
                return loss, metric_scores

        def evaluate(train_dataset, test_dataset, **metrics):
            """Trains the model and accumulates metrics on the test set
            on every epoch.
            """

            model.to(device)

            # Construct data loaders
            train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
            test_loader = DataLoader(dataset=test_dataset, batch_size=1)

            # Setup training
            optimiser = opt(model.parameters(), lr=lr)
            
            train_losses = []
            val_losses = []
            val_metrics = {metric: [] for metric in metrics}

            epoch_progress = range(1, num_epochs + 1)
            if not detailed_log:
                epoch_progress = tqdm(epoch_progress, desc='Epoch')

            for epoch in epoch_progress:
                if detailed_log:
                    header = f'Epoch {epoch}'
                    print(header)
                    print('=' * len(header))

                progress = train_loader
                if detailed_log:
                    progress = tqdm(progress,
                                    desc=f'Training: Mini-Batch SGD (size={batch_size})')
                    
                model.train()
                for X, scores in progress:
                    optimiser.zero_grad()

                    pred = model(*(x.to(device) for x in X)).squeeze()
                
                    loss = loss_fn(pred, scores.to(device))

                    loss.backward()
                    optimiser.step()
                
                # Only append most recent loss
                train_losses.append(loss.cpu().detach().numpy())
                
                if detailed_log:
                    print(f'Most recent loss:', loss.cpu().detach().numpy())
                    print()

                # Validation
                preds = []
                scores = []
                with torch.no_grad():
                    model.eval()

                    eval_progress = test_loader
                    if detailed_log:
                        eval_progress = tqdm(eval_progress, desc='Validating')

                    for X, score in eval_progress:
                        pred = model(*(x.to(device) for x in X)).squeeze().cpu()
                        preds.append(pred)
                        scores.append(score)
                
                    preds = torch.stack(preds)
                    scores = torch.cat(scores)

                    loss = loss_fn(preds, scores)
                    metric_scores = {name: metric_fn(preds, scores)
                                    for name, metric_fn in metrics.items()}

                    val_losses.append(loss.cpu().detach().numpy())
                    for name, metric in metric_scores.items():
                        val_metrics[name].append(metric)
            
            return model, (train_losses, val_losses, val_metrics, preds, scores)

        model.fit = fit
        model.predict = predict
        model.evaluate = evaluate

        return model

    return factory


### Cross-Validation

In [0]:
from sklearn.model_selection import KFold

def torch_kfold_cross_validate(make_model, *,
                               inputs,
                               outputs,
                               n_splits=2,
                               random_state=0,
                               **metrics):

    # Convert output scores to PyTorch tensor.
    if not isinstance(outputs, torch.FloatTensor):
        outputs = torch.FloatTensor(outputs)
    
    if not isinstance(inputs, list):
        inputs = [inputs]
    
    # Reset seeding
    torch.manual_seed(random_state)
    if device == 'cuda':
        torch.cuda.manual_seed(random_state)
    np.random.seed(random_state)

    cv_split = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    total_loss = 0
    total_metrics = {metric: 0 for metric in metrics}

    for fold, (train_idxs, test_idxs) in enumerate(cv_split.split(inputs[0])):
        print(f'>> Fold {fold + 1}')

        # Create new model.
        model = make_model()

        train_set = build_dataset(X=inputs, Y=outputs, idxs=train_idxs)
        test_set = build_dataset(X=inputs, Y=outputs, idxs=test_idxs)

        # Training
        model.fit(train_set)

        # Evaluation
        loss, metric_scores = model.predict(test_set, **metrics)

        total_loss += loss
        total_metrics = {name: total_metrics[name] + metric_scores[name]
                         for name in metrics}

        print()

    total_loss /= n_splits
    total_metrics = {name: total_metrics[name] / n_splits
                     for name in metrics}
    return total_loss, total_metrics

### Visualisations

In [0]:
def performance_to_dataframe(performance, *,
                             sort_by=['pearson', 'Loss'],
                             ascending=[0, 1]):
    performance_np = np.array(performance)
    performance_df = pd.DataFrame(data=performance_np[1:], columns=performance_np[0])
    return performance_df.sort_values(by=sort_by, ascending=ascending)

In [0]:
# Mount Google Drive to save data

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # FOR COLAB
    # =========
    from google.colab import drive
    drive.mount('/content/gdrive')

    !ls '/content'

    def in_gdrive(path):
        return os.path.join('/content/gdrive/My Drive/Colab Notebooks', path)

In [0]:
def visualise(train_losses, val_losses, metrics, predicted, actual, *,
              name,
              save_to_gdrive=True):
    print('>> ACTUAL / PREDICTED SCORE ')
    plt.scatter(actual, predicted)
    plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'k--', lw=2)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')

    path = f'{name}_actual_predicted.png'
    if save_to_gdrive:
        path = in_gdrive(path)
    plt.savefig(path)
    plt.show()


    epoch_ticks = np.arange(1, len(train_losses) + 1, 1)
    print('>> TRAINING / VALIDATION LOSS ')
    plt.plot(epoch_ticks, train_losses, label='Training')
    plt.plot(epoch_ticks, val_losses, label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('RMSE')
    plt.legend(loc='best')
    plt.title(f'Final Training Loss: {train_losses[-1]}\n'
              f'Final Validation Loss: {val_losses[-1]}')

    path = f'{name}_train_val_loss.png'
    if save_to_gdrive:
        path = in_gdrive(path)
    plt.savefig(path)
    plt.show()


    print('\n>> VALIDATION METRICS')
    fig_size = 5
    fig, axes = plt.subplots(1, len(metrics), figsize=(fig_size, fig_size * len(metrics)))
    if not isinstance(axes, list):
        axes = [axes]

    for ax, (metric_name, metric_scores) in zip(axes, metrics.items()):
        ax.plot(epoch_ticks, metric_scores)
        ax.set_xlabel('Epochs')
        ax.set_ylabel(metric_name)
        ax.set_title(f'Final {metric_name}: {metric_scores[-1]}')

        print(f'Final {metric_name}: {metric_scores[-1]}')
    path = f'{name}_metrics.png'
    if save_to_gdrive:
        path = in_gdrive(path)
    fig.savefig(path)
    fig.show()

## Building Blocks

### Preprocessing

#### English

* spaCy - tokenisation
* NLTK - stop word removal

In [0]:
# Downloading spacy models for English

!spacy download en_core_web_md
!spacy link en_core_web_md en300 --force

# Downloading stop words for English

from nltk import download
from nltk.corpus import stopwords

download('stopwords')
stop_words_en = set(stopwords.words('english'))

In [0]:
# Get tokenizer

import spacy

nlp_en = spacy.load('en300')

def preprocess_en(sentence=None, *, keep_stopwords=False):
    """Preprocess English sentence using spaCy for tokenisation.
    Toggle `keep_stopwords=True` to preserve stopwords."""

    def wrapper(sentence):
        text = sentence.lower()
        processed = [token.lemma_ for token in nlp_en.tokenizer(text)]
        processed = [token for token in processed if token.isalpha()]
        if not keep_stopwords:
            processed = [token for token in processed if token not in stop_words_en]
        return processed

    return wrapper if sentence is None else wrapper(sentence)

#### Chinese

* Jieba - tokenisation
* @Tony607/Chinese_sentiment_analysis - stop words removal

In [0]:
# Download stopwords
FILE_STOP_WORDS_ZH = './chinese_stop_words.txt'

if not os.path.exists(FILE_STOP_WORDS_ZH):
    !wget -c https://github.com/Tony607/Chinese_sentiment_analysis/blob/master/data/chinese_stop_words.txt

with open(FILE_STOP_WORDS_ZH, 'r', encoding='utf-8') as f:
    stop_words_zh = [line.rstrip() for line in f]

In [0]:
import jieba

def preprocess_zh(sentence=None, *, keep_stopwords=False):
    """Preprocess Chinese sentence using jieba for tokenisation.
    Toggle `keep_stopwords=True` to preserve stopwords."""
    
    def wrapper(sentence):
        tokens = jieba.cut(sentence, cut_all=False)
        processed = [token for token in tokens if token.isalnum()]
        if not keep_stopwords:
            processed = [token for token in processed if token not in stop_words_zh]
        return processed

    return wrapper if sentence is None else wrapper(sentence)

### Pretrained Word Embeddings

#### English

* GloVe

In [0]:
import torchtext

glove = torchtext.vocab.GloVe(name='6B', dim=100)

def get_en_sentence_vector(tokens, *, dim=100):
    vectors = []
    for token in tokens:
        try:
            vectors.append(glove.vectors[glove.stoi[token]])
        except KeyError:
            pass

    if not vectors:
        vectors.append(torch.zeros(dim))
    
    return torch.stack(vectors)

#### Chinese

* NLPL

In [0]:
ZH_MODEL_BIN = 'model.bin'

if not os.path.exists(ZH_MODEL_BIN):
    !wget -O zh.zip http://vectors.nlpl.eu/repository/20/35.zip
    !unzip zh.zip 

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

wv_from_bin = KeyedVectors.load_word2vec_format(ZH_MODEL_BIN, binary=True) 

def get_zh_sentence_vector(tokens, *, dim=100):
    vectors = []
    for token in tokens:
        try:
            vectors.append(torch.Tensor(wv_from_bin[token]))
        except KeyError:
            pass

    if not vectors:
        vectors.append(torch.zeros(dim))
    
    return torch.stack(vectors)

### Sentence Representation

In [0]:
def pad_sentences(sents, *, pad_token=0):
    """Pad sentences with `pad_token` to the longest sentence in `sents."""

    # Get max sentence length
    sent_lengths = [len(sent) for sent in sents]
    max_sent_len = max(sent_lengths)
    
    # Create empty matrix with padding tokens
    padded_sents = np.ones((len(sents), max_sent_len)) * pad_token

    # Copy over the sequences
    for i, (sent_len, sent) in enumerate(zip(sent_lengths, sents)):
        padded_sents[i, 0:sent_len] = sent[:sent_len]
    return padded_sents

def average_vector(sent, *, dim=100):
    return torch.mean(sent, dim=0)

### Vocabulary Representation

In [0]:
class Language(object):

    PAD_TOKEN = '<PAD>'
    UNK_TOKEN = '<UNK>'

    def __init__(self, name):
        self.name = name
        self.word2idx = {}
        self.idx2word = {0: self.PAD_TOKEN,
                         1: self.UNK_TOKEN}

    def __len__(self):
        return len(self.idx2word)

    def add_sentence(self, sentence):
        for token in sentence:
            self.add_word(token)

    def add_word(self, word):
        if word not in self.word2idx:
            idx = len(self)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
    
    def sent_to_idxs(self, sent):
        return [self.word2idx.get(word, 1) for word in sent]

    def __getitem__(self, key):
        if isinstance(key, int):
            return self.idx2word[key]
        if isinstance(key, str):
            return self.word2idx[key]
        raise KeyError(key)
    
    def __repr__(self):
        return f'Language(name={self.name}) with {len(self)} words'

### BERT (`pytorch-pretrained-bert`)

In [0]:
!pip install pytorch-pretrained-bert

from pytorch_pretrained_bert import BertConfig, BertTokenizer, BertForSequenceClassification, BertModel
from pytorch_pretrained_bert.optimization import BertAdam

#### Tokenizer

In [0]:
def sentence_groups_to_bert_tokens(tokenizer, *sents, max_length=256):
    assert len(sents) > 0, 'No sentences to tokenise!'

    bert_inputs = []
    num_reserved_markers = 3

    for sent_group in zip(*sents):
        sent_tokens = [tokenizer.tokenize(sent) for sent in sent_group]
        total_length = sum([len(sent) for sent in sent_tokens]) + num_reserved_markers
        if total_length > max_length:
            raise Exception(f'Too long ({total_length} > {max_length})')

        tokens = ['[CLS]']
        for sent in sent_tokens:
            tokens += sent
            tokens.append('[SEP]')
        
        ids = tokenizer.convert_tokens_to_ids(tokens)

        # Apply sentence length padding
        padding = [0] * (max_length - len(ids))
        ids_tensor = torch.LongTensor(ids + padding)
        bert_inputs.append(ids_tensor)

    return torch.stack(bert_inputs)

#### Language Model

In [0]:
import math

def get_bert_embeddings(bert_lm, input_ids, *, batch_size=25):
    bert_lm.to(device)

    num_batches = math.ceil(len(input_ids) / batch_size)

    embs = []
    for batch_id in tqdm(range(num_batches), desc='Batch'):
        start_id = batch_id * batch_size
        end_id = (batch_id + 1) * batch_size
        input_id_batch = input_ids[start_id:end_id]

        if isinstance(bert_lm, BertForSequenceClassification):
            emb = bert_lm(input_id_batch.to(device))
        else:
            _, emb = bert_lm(input_id_batch.to(device))
        embs.append(emb.detach().cpu())

    return embs

### BERT (`transformers`)

In [0]:
!pip install transformers

from transformers import BertForSequenceClassification, BertTokenizer

In [0]:
import math

def get_bert_embeddings_with_mask(bert_lm, input_ids, attention_masks, *, batch_size=25):
    bert_lm.to(device)

    num_batches = math.ceil(len(input_ids) / batch_size)

    embs = []
    for batch_id in tqdm(range(num_batches), desc='Batch'):
        start_id = batch_id * batch_size
        end_id = (batch_id + 1) * batch_size

        input_id_batch = torch.LongTensor(input_ids[start_id:end_id])
        mask_batch = torch.LongTensor(attention_masks[start_id:end_id])

        _, emb = bert_lm(input_id_batch.to(device), mask_batch.to(device))
        embs.append(emb.detach().cpu())

    return embs

## Models

### 1) Baseline with FFNN Regressor

#### Pipeline

1. Manual preprocessing
    * EN - tokenisation with [spaCy](https://spacy.io),
    stopword removal
    * ZH - tokenisation with [jieba](https://github.com/fxsjy/jieba),
    stopword removal, 
2. Pretrained embeddings
    * EN - GloVe
    * ZH - TODO
3. Regression model
    * SVR
    * LinearRegression

In [0]:
##########
# ENGLISH
##########

# Preprocessing
preprocess_english = preprocess_en(keep_stopwords=False)
train_en_sents = [preprocess_english(sent) for sent in train_en]
val_en_sents = [preprocess_english(sent) for sent in val_en]
test_en_sents = [preprocess_english(sent) for sent in test_en]

# Get sentence average vector representation 
train_en_vecs = torch.stack([average_vector(get_en_sentence_vector(sent))
                             for sent in train_en_sents])
val_en_vecs = torch.stack([average_vector(get_en_sentence_vector(sent))
                           for sent in val_en_sents])
test_en_vecs = torch.stack([average_vector(get_en_sentence_vector(sent))
                            for sent in test_en_sents])

In [0]:
##########
# CHINESE
##########

# Preprocessing
preprocess_chinese = preprocess_zh(keep_stopwords=False)
train_zh_sents = [preprocess_chinese(sent) for sent in train_zh]
val_zh_sents = [preprocess_chinese(sent) for sent in val_zh]
test_zh_sents = [preprocess_chinese(sent) for sent in test_zh]

# Get sentence average vector representation
train_zh_vecs = torch.stack([average_vector(get_zh_sentence_vector(sent))
                             for sent in train_zh_sents])
val_zh_vecs = torch.stack([average_vector(get_zh_sentence_vector(sent))
                           for sent in val_zh_sents])
test_zh_vecs = torch.stack([average_vector(get_zh_sentence_vector(sent))
                            for sent in test_zh_sents])

In [0]:
train_concat_vecs = concat_zip(train_en_vecs, train_zh_vecs)
val_concat_vecs = concat_zip(val_en_vecs, val_zh_vecs)
test_concat_vecs = concat_zip(test_en_vecs, test_zh_vecs)

#### Model Architecture

Feed-forward neural network for regression

In [0]:
class FFNNRegression(nn.Module):

    def __init__(self, *, hidden_dims, input_dim, nonlin=F.relu):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.nonlin = nonlin

        prev_dim = input_dim
        for i, hidden_dim in enumerate(self.hidden_dims):
            setattr(self, f'hidden_{i}', nn.Linear(prev_dim, hidden_dim))
            prev_dim = hidden_dim
        
        self.out = nn.Linear(prev_dim, 1)

    def forward(self, x):
        act = x
        for i, _ in enumerate(self.hidden_dims):
            layer = getattr(self, f'hidden_{i}')
            act = self.nonlin(layer(act))
        return self.out(act)

#### Model Selection

In [0]:
#########################
# OPTIONS FOR GRID SEARCH
#########################
LR = 1e-2, 1e-3
NONLIN = F.relu, F.leaky_relu, torch.sigmoid
LAYERS = [], (16,), (128,), (32, 16), (64, 16), (128, 64, 16)

# Base configurations
non_loss_metrics = {'pearson': pearson_score}
base_hyperparameters = {'opt': torch.optim.Adam,
                        'num_epochs': 200,
                        'batch_size': 1,
                        'loss_fn': RMSELoss}

n_splits = 10
performance = []
for lr in LR:
    for nonlin in NONLIN:
        for hidden_dims in LAYERS:
            header = f'LR={lr}, nonlin={nonlin.__name__}, hidden_dims={hidden_dims}'
            print(header)
            print('=' * len(header))

            model_params = dict(input_dim=200, hidden_dims=hidden_dims, nonlin=nonlin)
            hyperparameters = dict(**base_hyperparameters)
            hyperparameters['lr'] = lr

            model_factory = torch_model_factory(FFNNRegression,
                                                model_params,
                                                **hyperparameters,
                                                detailed_log=False)
            
            loss, metrics = torch_kfold_cross_validate(model_factory,
                                                       inputs=train_concat_vecs,
                                                       outputs=train_scores,
                                                       n_splits=n_splits,
                                                       **non_loss_metrics)
            
            performance.append(((lr, nonlin.__name__, hidden_dims), (loss, metrics)))
            print()

for (lr, nonlin, hidden_dims), (loss, metrics) in performance:
    print(f'LR={lr}, nonlin={nonlin}, hidden_dims={hidden_dims}')
    print(f'  MAE Loss:\t {loss}')
    for name, metric in metrics.items():
        print(f'  {name}:\t {metric}')
    print()

#### Validation

In [0]:
# Pick the optimal model hyperparameters to train.
model_params = dict(input_dim=200, hidden_dims=(100,),
                    nonlin=torch.tanh)

hyperparameters = {'opt': torch.optim.Adam,
                   'num_epochs': 500,
                   'batch_size': 16,
                   'lr': 1e-4,
                   'loss_fn': RMSELoss}

model_for_validation = torch_model_factory(FFNNRegression,
                                           model_params,
                                           **hyperparameters,
                                           detailed_log=False)()

train_set = build_dataset(X=[train_concat_vecs], Y=train_scores)
val_set = build_dataset(X=[val_concat_vecs], Y=val_scores)

non_loss_metrics = {'pearson': pearson_score}

print('>> TRAINING START')
model, results = model_for_validation.evaluate(train_set, val_set, **non_loss_metrics)
train_losses, val_losses, metrics, predicted, actual = results
print('>> TRAINING FINISH')

In [0]:
visualise(train_losses, val_losses, metrics, predicted, actual,
          name='BERT sentence pair embeddings',
          save_to_gdrive=False)

### 2) Autoencoder with Quality Estimation Vectors

#### Pipeline

1. Manual preprocessing
    * EN - tokenisation with [spaCy](https://spacy.io),
    stopword removal
    * ZH - tokenisation with [jieba](https://github.com/fxsjy/jieba),
    stopword removal, 
2. Model

In [0]:
##########
# ENGLISH
##########

preprocess_english = preprocess_en(keep_stopwords=False)
train_en_sents = [preprocess_english(sent) for sent in train_en]
val_en_sents = [preprocess_english(sent) for sent in val_en]
test_en_sents = [preprocess_english(sent) for sent in test_en]

EN = Language('EN')
for sent in train_en_sents:
    EN.add_sentence(sent)
print(EN)

print()
print('Sample sentence')
sample_sent_en = train_en_sents[42]
print(sample_sent_en)
print(EN.sent_to_idxs(sample_sent_en))

train_en_idxs = pad_sentences([EN.sent_to_idxs(sent) for sent in train_en_sents])
val_en_idxs = pad_sentences([EN.sent_to_idxs(sent) for sent in val_en_sents])
test_en_idxs = pad_sentences([EN.sent_to_idxs(sent) for sent in test_en_sents])

In [0]:
##########
# CHINESE
##########

preprocess_chinese = preprocess_zh(keep_stopwords=False)
train_zh_sents = [preprocess_chinese(sent) for sent in train_zh]
val_zh_sents = [preprocess_chinese(sent) for sent in val_zh]
test_zh_sents = [preprocess_chinese(sent) for sent in test_zh]

ZH = Language('ZH')
for sent in train_zh_sents:
    ZH.add_sentence(sent)
print(ZH)

print()
print('Sample sentence')
sample_sent_zh = train_zh_sents[0]
print(sample_sent_zh)
print(ZH.sent_to_idxs(sample_sent_zh))

train_zh_idxs = pad_sentences([ZH.sent_to_idxs(sent) for sent in train_zh_sents])
val_zh_idxs = pad_sentences([ZH.sent_to_idxs(sent) for sent in val_zh_sents])
test_zh_idxs = pad_sentences([ZH.sent_to_idxs(sent) for sent in test_zh_sents])

#### Model Architecture

* Embedding layers
* RNN autoencoder to generate quality estimation vectors
* Output regression layer using FFNN

In [0]:
class AutoencoderQEV(nn.Module):

    def __init__(self, *, en_vocab_size, zh_vocab_size, emb_dim):
        super().__init__()
        self.en_vocab_size = en_vocab_size
        self.zh_vocab_size = zh_vocab_size
        self.emb_dim = emb_dim

        """Source sentence: embedding layer + encoder."""
        self.source_embedding = nn.Embedding(self.en_vocab_size, self.emb_dim)
        self.source_rnn = nn.GRU(input_size=self.emb_dim,
                                 hidden_size=self.emb_dim,
                                 bidirectional=True)

        """Target sentence: embedding layer + decoder with attention."""
        self.target_embedding = nn.Embedding(self.zh_vocab_size, self.emb_dim)
        self.target_rnn = nn.GRU(input_size=self.emb_dim,
                                 hidden_size=self.emb_dim * 2,
                                 bidirectional=False)

        """RNN for producing summary unit."""
        self.qualvec_rnn = nn.GRU(input_size=self.emb_dim * 2,
                                  hidden_size=self.emb_dim,
                                  bidirectional=False)

        """Regression output layer."""
        self.regressor_output = nn.Linear(in_features=self.emb_dim,
                                          out_features=1)

    def forward(self, en_sent, zh_sent, *, log=suppress_log, get_qualvecs=False):
        """Perform forward pass and returns the prediction scores.

        Parameters:
            en_sent: (batch_size, en_max_sent_len)
            zh_sent: (batch_size, zh_max_sent_len)
        
        Debug parameters:
            log: custom `print` function, defaults to suppressing messages
            get_qualvecs: if True, returns the quality vectors instead.
        """

        en_batch_size, en_sent_len = en_sent.shape
        en_emb = self.source_embedding(en_sent)
        log('en_emb:', en_emb.shape)

        en_emb = en_emb.view(en_sent_len, en_batch_size, -1)
        log('en_emb:', en_emb.shape)
        en_all_hids, en_last_hid = self.source_rnn(en_emb)

        log('en_all_hids:', en_all_hids.shape)
        log('en_last_hid:', en_last_hid.shape)

        ############################################
        def get_context(prev_state):
            log('prev_state:', prev_state.shape)
            s_s = []
            for hid in en_all_hids:
                s_s_batches = torch.Tensor([
                    one_hid_batch.dot(one_prev_state_batch)
                    for one_hid_batch, one_prev_state_batch in zip(prev_state, hid)
                ])
                s_s.append(s_s_batches)
            
            s_s = torch.stack(s_s, dim=0)
            log('s_s', s_s.shape)

            a_s = F.softmax(s_s, dim=0)
            log('a_s', a_s.shape)

            ctx_vecs = []
            for j, (a_i, hid) in enumerate(zip(a_s, en_all_hids)):
                vecs = []
                for i, (one_a_batch, one_hid_batch) in enumerate(zip(a_i, hid)):
                    vec = one_a_batch * one_hid_batch
                    vecs.append(vec)
                
                vecs = torch.stack(vecs)
                ctx_vecs.append(vecs)

            ctx_vecs = torch.stack(ctx_vecs).sum(dim=0)
            log(f'ctx_vecs', ctx_vecs.shape)
            return ctx_vecs

        ############################################

        zh_batch_size, zh_sent_len = zh_sent.shape
        log('zh_sent_len', zh_sent_len)
        zh_emb = self.target_embedding(zh_sent)

        log('zh_emb:', zh_emb.shape)
        zh_emb = zh_emb.view(zh_sent_len, zh_batch_size, -1)
        log('zh_emb:', zh_emb.shape)

        qualvecs = []
        zh_hid = None
        for zh in zh_emb:
            log('zh:', zh.shape)
            zh = zh.view(1, zh_batch_size, -1)
            log('zh:', zh.shape)
            if zh_hid is None:
                _, zh_hid = self.target_rnn(zh)
            else:
                _, zh_hid = self.target_rnn(zh, zh_hid)
            log('zh_hid:', zh_hid.shape)

            zh_hid_reshaped = zh_hid.view(zh_batch_size, -1)
            log('zh_hid_reshaped:', zh_hid_reshaped.shape)

            ctx = get_context(zh_hid_reshaped)
            log('ctx:', ctx.shape)
            
            # Linear combination = sum
            qualvecs.append(ctx + zh_hid_reshaped)

        qualvecs = torch.stack(qualvecs)
        log('qualvecs:', qualvecs.shape)

        if get_qualvecs:
            return qualvecs

        _, qualvec_hid = self.qualvec_rnn(qualvecs)
        log('qualvec_hid:', qualvec_hid.shape)

        qualvec_hid = qualvec_hid.view(zh_batch_size, -1)
        log('qualvec_hid:', qualvec_hid.shape)

        qualvec_hid_act = torch.tanh(qualvec_hid)


        score = self.regressor_output(qualvec_hid_act)
        log('score', score.shape)
        
        return score

#### Model Selection

In [0]:
non_loss_metrics = {'pearson': pearson_score}

base_hyperparameters = {'opt': torch.optim.Adam,
                        'num_epochs': 15,
                        'loss_fn': RMSELoss}

LR = 1e-3, 1e-4
BATCH_SIZES = 12, 8

performance = [['Model', 'Loss'] + [name for name in non_loss_metrics]]
for lr in LR:
    for batch_size in BATCH_SIZES:
        model_params = dict(en_vocab_size=len(EN),
                            zh_vocab_size=len(ZH),
                            emb_dim=100)
        
        hyperparameters = dict(**base_hyperparameters)
        hyperparameters['lr'] = lr
        hyperparameters['batch_size'] = batch_size

        model_factory = torch_model_factory(AutoencoderQEV, model_params,
                                            **hyperparameters,
                                            detailed_log=True)

        loss, metrics = torch_kfold_cross_validate(model_factory,
                                    inputs=[train_en_idxs, train_zh_idxs],
                                    outputs=train_scores,
                                    n_splits=2,
                                    **non_loss_metrics)
        
        performance.append([f'LR={lr}, batch_size={batch_size}', loss.item()] + list(metrics.values()))

performance_to_dataframe(performance)
dataframe

#### Validation

In [0]:
opt_hyperparameters = {'opt': torch.optim.Adam,
                       'lr': 1e-3,
                       'num_epochs': 15,
                       'batch_size': 1,
                       'loss_fn': RMSELoss}

non_loss_metrics = {'pearson': pearson_score}

model_for_validation = torch_model_factory(AutoencoderQEV,
                                           dict(en_vocab_size=len(EN), zh_vocab_size=len(ZH), emb_dim=100),
                                           **opt_hyperparameters,
                                           detailed_log=True)()

train_set = build_dataset(X=[train_en_idxs, train_zh_idxs], Y=train_scores)
val_set = build_dataset(X=[val_en_idxs, val_zh_idxs], Y=val_scores)

model, results = model_for_validation.evaluate(train_set, val_set, **non_loss_metrics)
train_losses, val_losses, metrics, predicted, actual = results

In [0]:
visualise(train_losses, val_losses, metrics, predicted, actual,
          name='Autoencoder with Quality Estimation Vectors',
          save_to_gdrive=True)

### 3) BERT with Sentence Score

#### Pipeline

1. BertTokenizer to prepare inputs for BERT
2. BertForSequenceClassification to obtain sentence scores
3. Regression model to obtain QE scores

In [0]:
# Load tokenisers
print(f'Loading EN tokenizer...')
en_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
print('done!')
print()

print(f'Loading ZH tokenizer...')
zh_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
print('done!')
print()


# Check tokenizers
sample_sent_id = 42
print('English')
print(train_en[sample_sent_id])
print(en_tokenizer.tokenize(train_en[sample_sent_id]))
print()

print('Chinese')
print(train_zh[sample_sent_id])
print(zh_tokenizer.tokenize(train_zh[sample_sent_id]))

In [0]:
# Convert sentences to BERT tokens
train_en_inputs = sentence_groups_to_bert_tokens(en_tokenizer, train_en, max_length=128)
val_en_inputs = sentence_groups_to_bert_tokens(en_tokenizer, val_en, max_length=128)
test_en_inputs = sentence_groups_to_bert_tokens(en_tokenizer, test_en, max_length=128)

train_zh_inputs = sentence_groups_to_bert_tokens(zh_tokenizer, train_zh, max_length=128)
val_zh_inputs = sentence_groups_to_bert_tokens(zh_tokenizer, val_zh, max_length=128)
test_zh_inputs = sentence_groups_to_bert_tokens(zh_tokenizer, test_zh, max_length=128)

In [0]:
# Get source sentence scores from pre-trained English model
en_bert = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=1)

train_en_embs = get_bert_embeddings(en_bert, train_en_inputs, batch_size=1)
val_en_embs = get_bert_embeddings(en_bert, val_en_inputs, batch_size=1)
test_en_embs = get_bert_embeddings(en_bert, test_en_inputs, batch_size=1)

In [0]:
# Get MT scores from pre-trained Chinese model
zh_bert = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=1)

train_zh_embs = get_bert_embeddings(zh_bert, train_zh_inputs)
val_zh_embs = get_bert_embeddings(zh_bert, val_zh_inputs)
test_zh_embs = get_bert_embeddings(zh_bert, test_zh_inputs)

In [0]:
# Concatenate all pairs of English-Chinese sentence scores
train_sent_scores = concat_zip(train_en_embs, train_zh_embs).numpy()
val_sent_scores = concat_zip(val_en_embs, val_zh_embs).numpy()
test_sent_scores = concat_zip(test_en_embs, test_zh_embs).numpy()

#### Model Architecture

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR

#### Model Selection

In [0]:
from sklearn.model_selection import cross_validate, KFold

def kfold_cross_validate_sklearn(model, train_inputs, train_scores, scoring,
                                 n_splits=10,
                                 random_state=0):
    if not isinstance(train_scores, np.ndarray):
        train_scores = np.array(train_scores)

    cv_split = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    result = cross_validate(model, train_inputs, train_scores,
                            cv=cv_split,
                            scoring=scoring)
    return result

In [0]:
from sklearn.metrics import mean_absolute_error

models_to_test = [('LinearRegression', LinearRegression()),
                  ('SVR(kernel=linear)', SVR(kernel='linear')),
                  ('SVR(kernel=rbf)', SVR(kernel='rbf')),
                  ('SVR(kernel=poly)', SVR(kernel='poly')),]

scoring = {'Pearson': pearson_sklearn,
           'MAE': make_scorer(mean_absolute_error)}

n_splits = 5
for name, model in models_to_test:
    print(name)
    print('=' * len(name))
    metrics = kfold_cross_validate_sklearn(model,
                                           train_sent_scores,
                                           train_scores,
                                           scoring,
                                           n_splits=n_splits)
    
    for name, scores in metrics.items():
        average = sum(scores) / n_splits
        print(f'  {name}\t: {average:.5f}')
    print()

#### Validation

From cross validation, we select `SVR(kernel=linear)` which had the lowest MAE
and approximately the highest Pearson score.

We use this to fit on the full training set and evaluate on the unseen validation set.

In [0]:
print(f'Fitting on training set of size {len(train_scores)}...', end='')
model = SVR(kernel='linear')
model.fit(train_sent_scores, train_scores)
print('done!')
print()

print(f'Predicting on validation set of size {len(val_sent_scores)}...', end='')
preds = model.predict(val_sent_scores)
print('done!')
print()

loss = RMSELoss(preds, val_scores)
pearson = pearson_score(preds, val_scores)
print(f'MAE Loss:\t {loss:.5f}')
print(f'Pearson:\t {pearson:.5f}')

predicted, actual = preds, np.array(val_scores)
plt.scatter(actual, predicted)
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

### 4) BERT with Sentence Pair Embeddings

#### Pipeline

1. BertTokenizer to prepare inputs for BERT
2. BertModel to obtain sentence _pair_ embeddings (default dimension = 768)
3. FFNN to obtain quality estimation scores

In [0]:
from transformers import BertTokenizer

# Load tokeniser
print(f'Loading tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',
                                          do_lower_case=False)
print('done!')
print()

# Check tokeniser
sample_sent_id = 42

print('English')
print(train_en[sample_sent_id])
print(tokenizer.tokenize(train_en[sample_sent_id]))
print()

print('Chinese')
print(train_zh[sample_sent_id])
print(tokenizer.tokenize(train_zh[sample_sent_id]))

In [0]:
train_inputs = tokenizer.batch_encode_plus(list(zip(train_en, train_zh)), pad_to_max_length=True)
val_inputs = tokenizer.batch_encode_plus(list(zip(val_en, val_zh)), pad_to_max_length=True)
test_inputs = tokenizer.batch_encode_plus(list(zip(test_en, test_zh)), pad_to_max_length=True)

In [0]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-multilingual-cased')

In [0]:
print('\n>> Training')
train_embeddings = get_bert_embeddings_with_mask(bert, train_inputs['input_ids'], train_inputs['attention_mask'])
train_embeddings_tensor = torch.cat(train_embeddings)

print('\n>> Validation')
val_embeddings = get_bert_embeddings_with_mask(bert, val_inputs['input_ids'], val_inputs['attention_mask'])
val_embeddings_tensor = torch.cat(val_embeddings)

print('\n>> Validation')
test_embeddings = get_bert_embeddings_with_mask(bert, test_inputs['input_ids'], test_inputs['attention_mask'])
test_embeddings_tensor = torch.cat(val_embeddings)

#### Model Architecture

In [0]:
class FFNNRegression(nn.Module):

    def __init__(self, *, hidden_dims, input_dim=768, nonlin=F.relu):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.nonlin = nonlin

        prev_dim = input_dim
        for i, hidden_dim in enumerate(self.hidden_dims):
            setattr(self, f'hidden_{i}', nn.Linear(prev_dim, hidden_dim))
            prev_dim = hidden_dim
        
        self.out = nn.Linear(prev_dim, 1)

    def forward(self, x):
        act = x
        for i, _ in enumerate(self.hidden_dims):
            layer = getattr(self, f'hidden_{i}')
            act = self.nonlin(layer(act))
        return self.out(act)

#### Model Selection

In [0]:
#########################
# OPTIONS FOR GRID SEARCH
#########################
LR = 1e-3, 1e-4
NONLIN = F.leaky_relu, torch.tanh,
LAYERS = [[]] + [[2 ** i] for i in range(1, 5)]

# Base configurations
non_loss_metrics = {'pearson': pearson_score}
base_hyperparameters = {'opt': torch.optim.Adam,
                        'num_epochs': 250,
                        'batch_size': 64,
                        'loss_fn': RMSELoss}

n_splits = 5
performance = [['Model', 'Loss'] + [name for name in non_loss_metrics]]
for lr in LR:
    for nonlin in NONLIN:
        for hidden_dims in LAYERS:
            header = f'LR={lr}, nonlin={nonlin.__name__}, hidden_dims={hidden_dims}'
            print(header)
            print('=' * len(header))

            model_params = dict(hidden_dims=hidden_dims, nonlin=nonlin)
            hyperparameters = dict(**base_hyperparameters)
            hyperparameters['lr'] = lr

            model_factory = torch_model_factory(FFNNRegression,
                                                model_params,
                                                **hyperparameters,
                                                detailed_log=False)
            
            loss, metrics = torch_kfold_cross_validate(model_factory,
                                                       inputs=train_embeddings_tensor,
                                                       outputs=train_scores,
                                                       n_splits=n_splits,
                                                       **non_loss_metrics)
            
            performance.append([header, loss.item()] + list(metrics.values()))
            print(performance[-1])
            print()

In [0]:
# Evaluate performance
dataframe = performance_to_dataframe(performance)
dataframe.to_csv(in_gdrive('bert_with_sentence_embeddings_kfold_epoch-250_batch-64.csv'))
dataframe

#### Validation

In [0]:
# Pick the optimal model hyperparameters to train.
model_params = dict(hidden_dims=[128, 16], input_dim=768,
                    nonlin=torch.tanh)

hyperparameters = {'opt': torch.optim.Adam,
                   'num_epochs': 25,
                   'batch_size': 64,
                   'lr': 1e-4,
                   'loss_fn': RMSELoss}

model_for_validation = torch_model_factory(FFNNRegression,
                                           model_params,
                                           **hyperparameters,
                                           detailed_log=False)()

train_set = build_dataset(X=[train_embeddings_tensor], Y=train_scores)
val_set = build_dataset(X=[val_embeddings_tensor], Y=val_scores)

non_loss_metrics = {'pearson': pearson_score}

print('>> TRAINING START')
model, results = model_for_validation.evaluate(train_set, val_set, **non_loss_metrics)
print('>> TRAINING FINISH')

train_losses, val_losses, metrics, predicted, actual = results
visualise(train_losses, val_losses, metrics, predicted, actual,
          name='BERT sentence pair embeddings 250 epoch 64 batch',
          save_to_gdrive=False)

#### Prepare CodaLab Submission

We fit a model to the training _and_ validation set
and obtain QE score predictions for the test set corpus.

In [0]:
# Train model on train/validation set
train_val_tensor = torch.cat((train_embeddings_tensor, val_embeddings_tensor))
train_val_scores = torch.cat((torch.FloatTensor(train_scores), torch.FloatTensor(val_scores))).squeeze()

train_and_val_set = build_dataset(X=[train_val_tensor], Y=train_val_scores)

model_params = dict(hidden_dims=[16], input_dim=768,
                    nonlin=torch.tanh)

hyperparameters = {'opt': torch.optim.Adam,
                   'num_epochs': 250,
                   'batch_size': 64,
                   'lr': 1e-4,
                   'loss_fn': RMSELoss}

model_for_test = torch_model_factory(FFNNRegression,
                                     model_params,
                                     **hyperparameters,
                                     detailed_log=False)()

print('>> TRAINING START')
model_for_test.fit(train_and_val_set)
print('>> TRAINING FINISH')

In [0]:
# Predictions on test set
test_set = build_test_dataset(X=[test_embeddings_tensor])
test_loader = DataLoader(test_set, batch_size=1)

model.to(device)
model.eval()

preds = []
with torch.no_grad():
    for X in tqdm(test_loader, 'Getting test set predictions'):
        pred = model(*(x.to(device) for x in X)).squeeze().cpu()
        preds.append(pred)
    preds = torch.stack(preds)

# Save ZIP file
save_predictions(preds, zip_name='BERT_sentence_pair_embeddings_final')